In [ ]:
import urllib.request
from bs4 import BeautifulSoup
import csv

def download(url):
    print('Downloading:', url)
    try:
        html = urllib.request.urlopen(url).read()
    except urllib.error.URLError as e:
        print('Download error:', e.reason)
        html = None
    return html

def getList(url):
    year = url.split('/')[-1].split('.')[0].split('-')[-1]
    html = download(url)
    soup = BeautifulSoup(html, 'html.parser')
    table = soup.find('table')
    rows = table.find_all('tr')
    list = []
    for row in rows:
        cols = row.find_all('td')
        ranking = cols[0].text.strip()
        artist = cols[1].text.strip()
        song = cols[2].text.strip()

        list.append((year, ranking, artist, song))
    
    return list

html = download('https://suomenvuosilistat.blogspot.com/')

soup = BeautifulSoup(html, 'html.parser')
all_items = []
for link in soup.find_all('a'):
    text = link.text.strip()
    if text.isnumeric():
        url = link.get('href')
        list = getList(url)
        all_items.extend(list)

csv.register_dialect('mydialect', delimiter=';', quoting=csv.QUOTE_ALL)
with open('list.csv', 'w', encoding='utf-8') as f:
    writer = csv.writer(f, dialect='mydialect')
    for item in all_items:
        writer.writerow(item)